In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
#Creating Function To Extract Product Title
def get_title(soup):

    try:
        #Object name as title
        title = soup.find("span", attrs={"class":'VU-ZEz'})

        #convertig to text
        title_value = title.text

        #title as string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

#Function to extract product price
def get_price(soup):

    try:
        price = soup.find('div', attrs={"class":'Nx9bqj'}).text.strip()
        
    except AttributeError:
        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'yRaY8j'}).text.strip()
        except:
            price = ""
            
    return price
    
#Function to extract product Rating
def get_rating(soup):

    try:
        rating = soup.find('div', attrs={"class":'XQDdHH'}).text

    except AttributeError:
        try:
            rating = soup.find('span',attrs={"id":"productRating_LSTGMCGZCYPAFYBUNARLIR5LG_GMCGZCYPAFYBUNAR_"}).text.strip()
            
        except:
            rating = ""
        
    return rating

#function to extract number of user reviews
def get_review_count(soup):
    try:
        review_count = soup.find('span', attrs={"class":'Wphh3N'}).text.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_offers(soup):
    try:
        offers = soup.find("div", attrs={'class':"UkUFwK"}).text.strip()
    except AttributeError:
        offers = "Not Available"
    return offers

In [3]:
if __name__ == '__main__':

    #add your user agent
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0' , 'Accept-Language':'en-US, en;q=0.5'})

    #The webpage URL
    URL = "https://www.flipkart.com/search?q=samsung%20galaxy&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"

    #HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    #Soup Object containing all data
    soup = BeautifulSoup(webpage.content , 'html.parser')

     #Fetch links as List of Tag Objects
    links = soup.find_all("a", class_="wjcEIp")

    #store the links in list
    links_list = []

    #Creating Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))

    d = {'title':[], 'price':[], 'rating':[], 'reviews':[], 'offers':[]}

    #Creating Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get('https://flipkart.com' +link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content , 'html.parser')

        #Function calls to display all necessary product information 
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['offers'].append(get_offers(new_soup))

    flipkart_df = pd.DataFrame.from_dict(d)
    flipkart_df['title'] = flipkart_df['title'].replace('', np.nan)
    flipkart_df = flipkart_df.dropna(subset=['title'])
    flipkart_df.to_csv("flipkart_data.csv", header=True, index=False)

In [4]:
flipkart_df 

,title,price,rating,reviews,offers
0,"SAMSUNG Galaxy F06 5G (Lit Violet, 128 GB) (4...","₹9,999",4.2,606 Ratings & 55 Reviews,28% off
1,"SAMSUNG Galaxy F06 5G (Bahama Blue, 128 GB) (...","₹11,499",4,536 Ratings & 63 Reviews,25% off
2,"SAMSUNG Galaxy F05 (Twilight Blue, 64 GB) (4 ...","₹6,249",4.2,"38,098 Ratings & 2,226 Reviews",37% off
3,"SAMSUNG Galaxy M35 5G (DayBreak Blue, 128 GB) ...","₹14,370",4.4,"10,953 Ratings & 893 Reviews",41% off
4,SAMSUNG Intel Core Ultra 5 226V - (16 GB/512 G...,"₹1,14,990",4.7,(4),16% off
5,"SAMSUNG Galaxy F06 5G (Bahama Blue, 128 GB) (...","₹9,999",4.2,606 Ratings & 55 Reviews,28% off
6,SAMSUNG Intel Core Ultra 5 226V - (16 GB/512 G...,"₹1,31,990",4.7,(58),14% off
7,SAMSUNG Intel Core Ultra 7 258V - (32 GB/1 TB ...,"₹1,95,990",4.7,(58),16% off
8,"SAMSUNG Galaxy A05 (Light Green, 128 GB) (6 G...","₹8,950",4.2,640 Ratings & 26 Reviews,40% off
9,"SAMSUNG Galaxy A06 5G (Light Green, 128 GB) (...","₹13,999",4,,9% off


In [47]:
d

{'title': ['SAMSUNG Galaxy F05 (Twilight Blue, 64 GB)\xa0\xa0(4 GB RAM)',
  'SAMSUNG Galaxy M35 5G (DayBreak Blue, 128 GB)\xa0\xa0(6 GB RAM)',
  'SAMSUNG Galaxy A05 (Silver, 128 GB)\xa0\xa0(6 GB RAM)',
  'SAMSUNG Galaxy A05 (Light Green, 128 GB)\xa0\xa0(6 GB RAM)',
  'SAMSUNG Galaxy Tab A9+ 8 GB RAM 128 GB ROM 11.0 inch with Wi-Fi Only Tablet (Dark Blue)',
  'SAMSUNG Galaxy A05 (Silver, 64 GB)\xa0\xa0(4 GB RAM)',
  'SAMSUNG Galaxy S24 FE 5G (Blue, 128 GB)\xa0\xa0(8 GB RAM)',
  'SAMSUNG Galaxy F06 5G (Bahama Blue, 128 GB)\xa0\xa0(4 GB RAM)',
  'SAMSUNG Galaxy M35 5G (Thunder Grey, 256 GB)\xa0\xa0(8 GB RAM)',
  'SAMSUNG Galaxy S24+ 5G (Onyx Black, 256 GB)\xa0\xa0(12 GB RAM)',
  'SAMSUNG Galaxy M35 5G (Thunder Grey, 128 GB)\xa0\xa0(8 GB RAM)',
  'SAMSUNG Galaxy F06 5G (Bahama Blue, 128 GB)\xa0\xa0(6 GB RAM)',
  'SAMSUNG Galaxy A35 5G (Awesome Lilac, 256 GB)\xa0\xa0(8 GB RAM)',
  'SAMSUNG Galaxy M35 5G (Thunder Grey, 128 GB)\xa0\xa0(6 GB RAM)',
  'SAMSUNG Galaxy S24+ 5G (Cobalt Violet, 256